In [1]:
#!pip install "adapter-transformers@git+https://github.com/akufeldt/adapter-transformers.git@debug#egg=adapter-transformers&subdirectory=adapter-transformers"

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn import Softmax

from typing import List, Optional, Tuple, Union, Dict, Any

from datasets import load_dataset, Dataset, DatasetDict, load_metric, load_from_disk, concatenate_datasets
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, EarlyStoppingCallback
from transformers import PreTrainedModel, TrainingArguments, Trainer
#from transformers.adapters import AdapterTrainer

import pandas as pd
import numpy as np
import evaluate

import random
import math
import time
from tqdm import tqdm
import os
import json

/home/phonnego/miniconda3/envs/lrl_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
seed = 42
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
_numpy_rng = np.random.default_rng(seed)
random.seed(seed)
np.random.seed(seed)
torch.use_deterministic_algorithms(False)
os.environ['PYTHONHASHSEED'] = str(seed)

In [4]:
os.environ["WANDB_DISABLED"] = "true"

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
device

device(type='cuda')

# Load in model

In [7]:
model_name = 'm2m100_418M'
fine_tune_model_name = 'ha-en-finetune-og'
experiment = 'ha-en-finetune-og-0.7-improve-1'
dataset_name = 'data/ha-en'

In [8]:
model = M2M100ForConditionalGeneration.from_pretrained(f"Parikshith/{fine_tune_model_name}")
# model = torch.nn.DataParallel(model, device_ids=[2, 3, 4])
model = model.to(device)
tokenizer = M2M100Tokenizer.from_pretrained(f"facebook/{model_name}")

# Prepare data

In [9]:
src_lang = 'ha'
tgt_lang = 'en'
tokenizer.src_lang = "ha"
tokenizer.tgt_lang = "en"

In [13]:
# Load the datasets from Hugging Face Hub
original_train_dataset = load_dataset("pranjali97/ha-en_RL-grow1_train", split='train')
original_valid_dataset = load_dataset("pranjali97/ha-en_RL-grow1_valid", split='train')

new_filtered_train_dataset = original_train_dataset.filter(lambda example: example['score'] > 0.7)
new_filtered_valid_dataset = original_valid_dataset.filter(lambda example: example['score'] > 0.7)
new_filtered_train_dataset = new_filtered_train_dataset.remove_columns(['score', 'ref'])
new_filtered_valid_dataset = new_filtered_valid_dataset.remove_columns(['score', 'ref'])

train_dataset = load_dataset('csv', data_files='/home/phonnego/lrl_capstone/Language-Adapters/Data/en-ha/cleaned_train.csv', split='train')
valid_dataset = load_dataset('csv', data_files='/home/phonnego/lrl_capstone/Language-Adapters/Data/en-ha/cleaned_dev.csv', split='train')

train_dataset = train_dataset.rename_column('ha', 'src')
valid_dataset = valid_dataset.rename_column('ha', 'src')
train_dataset = train_dataset.rename_column('en', 'mt')
valid_dataset = valid_dataset.rename_column('en', 'mt')

new_train_dataset = concatenate_datasets([train_dataset, new_filtered_train_dataset])
new_valid_dataset = valid_dataset
new_train_dataset = new_train_dataset.shuffle(seed=42)
new_valid_dataset = new_valid_dataset.shuffle(seed=42)

/home/phonnego/miniconda3/envs/lrl_env/lib/python3.8/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/phonnego/miniconda3/envs/lrl_env/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [14]:
train_dataset

Dataset({
    features: ['mt', 'src'],
    num_rows: 9818
})

In [15]:
valid_dataset

Dataset({
    features: ['mt', 'src'],
    num_rows: 1113
})

In [16]:
new_train_dataset

Dataset({
    features: ['mt', 'src'],
    num_rows: 10620
})

In [17]:
new_valid_dataset

Dataset({
    features: ['mt', 'src'],
    num_rows: 1113
})

In [18]:
# Define the preprocess function
def preprocess_function(examples):
    inputs = examples['src']  # Hausa sentences
    targets = examples['mt']  # English translations
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=256, truncation=True, padding="max_length")
    # Return the tokenized inputs and labels
    return {'input_ids': model_inputs['input_ids'], 'attention_mask': model_inputs['attention_mask'], 'labels': labels['input_ids']}

# Apply the preprocess function to the datasets
new_tokenized_train_dataset = new_train_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src',  'mt']  # Specify the correct columns to remove
)
new_tokenized_valid_dataset = new_valid_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src',  'mt']  # Specify the correct columns to remove
)

# Create the DatasetDict
new_tokenized_dataset = DatasetDict({
    'train': new_tokenized_train_dataset,  # Directly assign the processed dataset
    'validation': new_tokenized_valid_dataset  # Directly assign the processed dataset
})

Map:   0%|          | 0/10620 [00:00<?, ? examples/s]/home/phonnego/miniconda3/envs/lrl_env/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 1113/1113 [00:00<00:00, 3096.98 examples/s]


In [19]:
new_tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10620
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1113
    })
})

# Training Setup

In [20]:
sacrebleu = evaluate.load("sacrebleu")
wer = evaluate.load("wer")

In [21]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    labels = eval_preds.label_ids
    pred_ids = eval_preds.predictions
    if isinstance(pred_ids, tuple):
        pred_ids = pred_ids[0]
    
    preds = np.argmax(pred_ids, axis=-1)

    # removeme
    #import warnings
    #warnings.warn(f"unprocessed preds: {preds[0]}\n)")
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True) 

    # removeme
    #warnings.warn(f"unprocessed decoded labels: {tokenizer.batch_decode(labels)[0]}\n)")

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # remove me
    #inputs = eval_preds.input_ids
    #decoded_inputs = tokenizer.batch_decode(inputs)
    
    # Removeme
    import warnings
    warnings.warn(f"preds: {decoded_preds[0]}\n)")
    warnings.warn(f"labels: {decoded_labels[0]}\n)")

    bleu_result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    metrics = {"bleu": bleu_result["score"]}

    flattened_decoded_labels = [' '.join([str(x) for x in l]) for l in decoded_labels]
    wer_score = wer.compute(predictions=decoded_preds, references=flattened_decoded_labels)
    metrics["wer"] = wer_score

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    metrics["gen_len"] = np.mean(prediction_lens)
    metrics = {k: round(v, 4) for k, v in metrics.items()}
    return metrics

In [22]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [23]:
training_args = Seq2SeqTrainingArguments(
    f"./rest/{experiment}/model",
    # evaluation_strategy="steps",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=15,
    warmup_steps=0,
    # lr_scheduler_type='cosine_with_restarts',
    # gradient_accumulation_steps=4,
    eval_accumulation_steps=22,
    # gradient_checkpointing=True,
    # predict_with_generate=True,
    fp16=True,
    do_train=True,
    do_eval=True,
    logging_steps=5,
    # eval_steps=5,
    save_strategy="epoch",
    metric_for_best_model="bleu",
    load_best_model_at_end=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=new_tokenized_dataset["train"],
    # train_dataset=new_tokenized_dataset_train,
    eval_dataset=new_tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    #optimizers=(optimizer, lr_scheduler),
    compute_metrics=compute_metrics,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [24]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Wer,Gen Len
1,0.062600,0.287041,31.423400,0.994700,35.245300
2,0.057500,0.291703,31.664000,0.995500,35.245300
3,0.040300,0.298681,32.432200,0.995400,35.245300
4,0.037500,0.305652,32.208600,0.995300,35.245300
5,0.033400,0.310058,32.576400,0.995200,35.245300
6,0.020300,0.316904,32.464400,0.994500,35.245300
7,0.022400,0.317964,32.496300,0.994900,35.244400
8,0.019400,0.325753,32.436500,0.995400,35.245300
9,0.016600,0.324033,32.926200,0.994900,35.245300
10,0.015300,0.331067,32.872600,0.995100,35.245300


/tmp/ipykernel_8747/3678268047.py:37: UserWarning: preds: Answer
)
  warnings.warn(f"preds: {decoded_preds[0]}\n)")
/tmp/ipykernel_8747/3678268047.py:38: UserWarning: labels: Answer
)
  warnings.warn(f"labels: {decoded_labels[0]}\n)")
/tmp/ipykernel_8747/3678268047.py:37: UserWarning: preds: Answer
)
  warnings.warn(f"preds: {decoded_preds[0]}\n)")
/tmp/ipykernel_8747/3678268047.py:38: UserWarning: labels: Answer
)
  warnings.warn(f"labels: {decoded_labels[0]}\n)")
/tmp/ipykernel_8747/3678268047.py:37: UserWarning: preds: Answer
)
  warnings.warn(f"preds: {decoded_preds[0]}\n)")
/tmp/ipykernel_8747/3678268047.py:38: UserWarning: labels: Answer
)
  warnings.warn(f"labels: {decoded_labels[0]}\n)")
/tmp/ipykernel_8747/3678268047.py:37: UserWarning: preds: Answer
)
  warnings.warn(f"preds: {decoded_preds[0]}\n)")
/tmp/ipykernel_8747/3678268047.py:38: UserWarning: labels: Answer
)
  warnings.warn(f"labels: {decoded_labels[0]}\n)")
/tmp/ipykernel_8747/3678268047.py:37: UserWarning: preds: An

TrainOutput(global_step=4980, training_loss=0.02430202059043818, metrics={'train_runtime': 5037.2286, 'train_samples_per_second': 31.625, 'train_steps_per_second': 0.989, 'total_flos': 8.63049407791104e+16, 'train_loss': 0.02430202059043818, 'epoch': 15.0})

In [25]:
base_model_path = f'./final_save/{experiment}'

if not os.path.exists(base_model_path):
    os.makedirs(base_model_path)

trainer.save_model(base_model_path)

# Eval finetuned model on test set

In [26]:
test_dataset = load_dataset('csv', data_files='/home/phonnego/lrl_capstone/Language-Adapters/Data/en-ha/test.csv', split='train')

#rename 'sentence_eng_Latn' to 'en' and 'sentence_hau_Latn' to 'ha'
test_dataset = test_dataset.rename_column('sentence_eng_Latn','mt')
test_dataset = test_dataset.rename_column('sentence_hau_Latn','src')

#tokenize the dataset
tokenized_test_dataset = test_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src',  'mt']  # Specify the correct columns to remove
)

#convert the dataset into dataset dict
tokenized_test_dataset = DatasetDict({
    'test': tokenized_test_dataset  # Directly assign the processed dataset
})

tokenized_test_dataset

Map:   0%|          | 0/1012 [00:00<?, ? examples/s]/home/phonnego/miniconda3/envs/lrl_env/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 1012/1012 [00:00<00:00, 2999.26 examples/s]


DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1012
    })
})

In [27]:
test_output = trainer.predict(tokenized_test_dataset["test"], forced_bos_token_id=tokenizer.get_lang_id(tgt_lang))

/tmp/ipykernel_8747/3678268047.py:37: UserWarning: preds: OnceWe are have fourmonth bablong babock un is not-diabetical and are to have withabetic," I says.
)
  warnings.warn(f"preds: {decoded_preds[0]}\n)")
/tmp/ipykernel_8747/3678268047.py:38: UserWarning: labels: "We now have 4-month-old mice that are non-diabetic that used to be diabetic," he added.
)
  warnings.warn(f"labels: {decoded_labels[0]}\n)")


In [28]:
test_output.metrics

{'test_loss': 0.48008817434310913,
 'test_bleu': 15.323,
 'test_wer': 0.9943,
 'test_gen_len': 34.2757,
 'test_runtime': 155.9771,
 'test_samples_per_second': 6.488,
 'test_steps_per_second': 0.814}

In [ ]:
new_eval_results = trainer.evaluate(tokenized_test_dataset["test"])

In [ ]:
new_eval_results

In [29]:
model.push_to_hub(experiment)

model.safetensors: 100%|██████████| 1.94G/1.94G [01:21<00:00, 23.7MB/s]


CommitInfo(commit_url='https://huggingface.co/Parikshith/ha-en-finetune-og-0.7-improve-1/commit/3ac7e15f7f83520d3cff66195fcd53707ab75a6d', commit_message='Upload M2M100ForConditionalGeneration', commit_description='', oid='3ac7e15f7f83520d3cff66195fcd53707ab75a6d', pr_url=None, pr_revision=None, pr_num=None)